Description

Embedding Model : text-embedding-3-small , sentence-transformers/LaBSE

Vector DB : MongoDB

โดยการทดสอบมีวิธีการที่แตกต่างกันสำหรับการทำ RAG และ การทำ Tokenize


In [81]:
import os
import pandas as pd
import time
import nest_asyncio
import asyncio
import numpy as np
from openai import AsyncOpenAI
from pymongo import MongoClient
from docx import Document
import tiktoken
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pathlib import Path
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModel
from PyPDF2 import PdfReader
from sklearn.decomposition import PCA
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import torch
from transformers import AutoTokenizer
from dotenv import load_dotenv


current_directory = os.getcwd()
print("Current Directory:", current_directory)  # พิมพ์ที่อยู่ปัจจุบัน

# ปรับเส้นทางไปยังไฟล์ .env ในโฟลเดอร์ venv
env_path = Path(current_directory).parent / 'venv' / '.env'
print("Env Path:", env_path)  # พิมพ์เส้นทางที่ไปยัง .env

load_dotenv(dotenv_path=env_path,override=True)


nest_asyncio.apply()

Current Directory: c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\test
Env Path: c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\.env


In [82]:
#KEY 
OPENAI_API_KEY : str 
EMBEDDING : str 
MONGO_URI : str


MONGO_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING = os.getenv("EMBEDDING")
client_openai = AsyncOpenAI(api_key=OPENAI_API_KEY)
mongo_client = MongoClient(MONGO_URI)
db = mongo_client["vector_db"]
collection = db["vectors"]

In [83]:
# file_path = "./data_csv_xlsx"
file_path = "./data_pdf"
# file_path = "./data_docx"
# 3. Define helper functions
def read_docx(path):
    doc = Document(path)
    text = []
    for para in doc.paragraphs:
        text.append(para.text.strip())  # เก็บข้อความทุกย่อหน้าลงใน list
    return text

def read_pdf(path):
    reader = PdfReader(path)
    pages = []
    for page in reader.pages:
        text = page.extract_text()
        if text:
            pages.append(text.strip())
    return pages



# Embedding By HuggingFace
# โหลดโมเดลและ tokenizer
# model_name = "bert-base-multilingual-cased"
# local_dir = "./models/bert-multi"  # ที่เก็บโมเดลแบบ local

# # # ดาวน์โหลด tokenizer และ model แล้วบันทึกไว้
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# tokenizer.save_pretrained(local_dir)
# model.save_pretrained(local_dir)

# # ใช้ GPU ถ้ามี
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# def embed_batch_bert(batch_texts):
#     encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
#     encoded_input = {key: val.to(device) for key, val in encoded_input.items()}
    
#     with torch.no_grad():
#         output = model(**encoded_input)
    
#     # ใช้ CLS token เป็น embedding (สามารถเปลี่ยนเป็น mean pooling ได้)
#     embeddings = output.last_hidden_state[:, 0, :]  # CLS token
#     return embeddings.cpu().numpy()

# async def batch_process_embedding_async(text_list, batch_size=32):
#     loop = asyncio.get_event_loop()
#     embeddings = []

#     for i in range(0, len(text_list), batch_size):
#         batch = text_list[i:i + batch_size]
#         # run embed_batch_bert in executor to simulate async
#         batch_embeddings = await loop.run_in_executor(None, embed_batch_bert, batch)
#         embeddings.extend(batch_embeddings)
#     return embeddings 

#GPT
async def embed_batch(batch, embed_model):
    response = await client_openai.embeddings.create(model=embed_model, input=batch)
    return [item.embedding for item in response.data]

async def batch_process_embedding_async(text_list, embed_model, batch_size=100):
    tasks = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        tasks.append(embed_batch(batch, embed_model))
    results = await asyncio.gather(*tasks)
    embeddings = [embedding for batch in results for embedding in batch]
    return embeddings

def cosine_similarity(vec1, vec2):
    vec1, vec2 = np.array(vec1), np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def count_tokens(text, model):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# ฟังก์ชันนับจำนวนโทเค็น
def count_tokens_2(text, model):
    # โหลด tokenizer สำหรับโมเดลที่กำหนด
    tokenizer = AutoTokenizer.from_pretrained(model)
    # ใช้ tokenizer ในการแปลงข้อความเป็นโทเค็นและนับจำนวนโทเค็น
    encoded = tokenizer.encode(text, truncation=True, padding=False)
    return len(encoded)

In [84]:
start_upload = time.perf_counter()
dfs = []
text_data = []

for filename in os.listdir(file_path):
    full_path = os.path.join(file_path, filename)
    
    if filename.endswith('.csv'):
        df = pd.read_csv(full_path)
        dfs.append(df)
        
    elif filename.endswith('.xlsx'):
        excel_data = pd.read_excel(full_path, sheet_name=None)
        for sheet in excel_data.values():
            dfs.append(sheet)
            
    elif filename.endswith('.docx'):
        text = read_docx(full_path)
        text_data.extend(text)
        
    elif filename.endswith('.pdf'):
        texts = read_pdf(full_path)
        text_data.extend(texts)

# รวมข้อมูลตามประเภท
if dfs:
    df_combined = pd.concat(dfs, ignore_index=True)
    df_combined.dropna(inplace=True)
    df_combined.drop_duplicates(inplace=True)
    df_combined.reset_index(drop=True, inplace=True)

elif text_data:
    df_combined = pd.DataFrame({"text": text_data})

end_upload = time.perf_counter()
print(f"✅ Upload เสร็จ {len(df_combined)} records ในเวลา {end_upload - start_upload:.2f} วินาที")
print("✅ Shape after cleansing:", df_combined.shape)
print(df_combined["text"])


✅ Upload เสร็จ 149 records ในเวลา 1.89 วินาที
✅ Shape after cleansing: (149, 1)
0      Fundamentals of Machine Learning \nand Analyzi...
1      Fundamentals of Machine Learning \nand Analyzi...
2      Fundamentals of Machine Learning \nand Analyzi...
3      คคานคา\nเทคโนโลยธีสารสนเทศสมบัยใหมม่ททาใหด้โลก...
4      สารบจัญ\n บทททท1 Jupyter Notebook ...............
                             ...                        
144    Fundamentals of Machine Learning and Analyzing...
145    138  ความรมตพพตนฐานทางด ตานการเรธยนรมตเครพพองจ...
146    Fundamentals of Machine Learning and Analyzing...
147    140  ความรมตพพตนฐานทางด ตานการเรธยนรมตเครพพองจ...
148    Fundamentals of Machine learning \nand Analyzi...
Name: text, Length: 149, dtype: object


In [85]:
texts = []
metadata_list = []
for i, row in df_combined.iterrows():
    metadata = row.to_dict()
    text = "\n".join([f"{k}: {v}" for k, v in metadata.items()])
    texts.append(text)
    metadata_list.append((f"vec-{i}", metadata))


In [86]:
start_embed = time.perf_counter()

embeddings = await batch_process_embedding_async(texts,EMBEDDING)

end_embed = time.perf_counter()
embed_time = end_embed - start_embed
print(f"✅ Embedding เสร็จ {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")

✅ Embedding เสร็จ 149 records ในเวลา 3.97 วินาที


In [87]:
start_upsert = time.perf_counter()

collection.delete_many({})  
documents = []
for (vec_id, metadata), embedding, raw_text in zip(metadata_list, embeddings, texts):
    documents.append({
        "_id": vec_id,
        "embedding": embedding,
        "metadata": metadata,
        "raw_text": raw_text
    })

collection.insert_many(documents)
print(f"✅ Inserted {len(documents)} documents into MongoDB.")

end_upsert = time.perf_counter()
upsert_time = end_upsert - start_upsert
print(f"✅ Upsertเสร็จ {len(documents)} records ในเวลา {upsert_time :.2f} วินาที")


✅ Inserted 149 documents into MongoDB.
✅ Upsertเสร็จ 149 records ในเวลา 0.08 วินาที


In [88]:
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")
print(f"✅ Upsert MongoDB เสร็จ {len(documents)} vectors ในเวลา {upsert_time :.2f} วินาที")
print(f"เวลาโดยรวมupload MongoDB ทั้งหมด : {embed_time+upsert_time:.2f} ")

✅ Embedding เสร็จทั้งหมด 149 records ในเวลา 3.97 วินาที
✅ Upsert MongoDB เสร็จ 149 vectors ในเวลา 0.08 วินาที
เวลาโดยรวมupload MongoDB ทั้งหมด : 4.04 


Pattern 1

In [89]:
async def retrieve_context_from_mongodb(question: str, top_k: int = 50):
    embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
    question_vector = embedder.embed_query(question)

    documents = list(collection.find())
    similarities = []
    for doc in documents:
        score = cosine_similarity(question_vector, doc["embedding"])
        similarities.append((score, doc))

    similarities.sort(reverse=True, key=lambda x: x[0])
    top_docs = [doc["raw_text"] for score, doc in similarities[:top_k]]
    return "\n".join(top_docs)

In [90]:
from tiktoken import encoding_for_model

# ใช้สำหรับการทดสอบอ่านไฟล์ 
def truncate_context(text, max_tokens, model="gpt-4"):
    enc = encoding_for_model(model)
    tokens = enc.encode(text)
    truncated = enc.decode(tokens[:max_tokens])
    return truncated

question = "ขอตัวอย่างการเปลี่ยนตำแหน่ง Legend "
context = await retrieve_context_from_mongodb(question)
short_context = truncate_context(context, max_tokens=512)
num_tokens_bf = count_tokens(context,model="text-embedding-3-small")
num_tokens_af = count_tokens(short_context,model="text-embedding-3-small")
print(f"Before จำนวน token ถูกตัด ทั้งหมดใน context: {num_tokens_bf}")
print(f"After จำนวน token หลังถูกตัด ทั้งหมดใน context: {num_tokens_af}")

Before จำนวน token ถูกตัด ทั้งหมดใน context: 29628
After จำนวน token หลังถูกตัด ทั้งหมดใน context: 512


In [91]:
# สร้าง PromptTemplate
prompt_template = PromptTemplate.from_template("""
ข้อมูลต่อไปนี้ถูกรวบรวมมาจากหลายแหล่งไฟล์ เช่น PDF, Word, Excel หรือ CSV ซึ่งอาจอยู่ในรูปแบบข้อความทั่วไปหรือเป็นข้อมูลเชิงตาราง:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- ไม่ต้องเขียนคำถามซ้ำ
- ไม่ต้องขึ้นต้นด้วยคำว่า "คำตอบ:"
- เริ่มต้นด้วยประโยค เช่น "จากเอกสารที่อ่าน..." หรือ "จากข้อมูลที่วิเคราะห์ได้..."
- จากนั้นจัดคำตอบให้อ่านง่ายในรูปแบบข้อ ๆ
- มีสรุปท้ายที่ใช้ถ้อยคำกระชับและไม่ซ้ำกับรายละเอียดด้านบน
- หลีกเลี่ยงการตอบซ้ำหรือลอกเนื้อหาเดิมซ้ำหลายรอบ

ตัวอย่างที่ 1:
จากข้อมูลที่วิเคราะห์ได้ สามารถสรุปเกี่ยวกับ Matplotlib ได้ดังนี้:  
1. เป็นไลบรารี่ในภาษา Python สำหรับสร้างภาพกราฟ 2 มิติ และ 3 มิติ  
2. รองรับการสร้างกราฟเส้น แท่ง วงกลม ฯลฯ  
3. ติดตั้งด้วยคำสั่ง pip install matplotlib  
4. เรียกใช้งานผ่าน import matplotlib.pyplot as plt  
5. ใช้งานง่ายและมีความยืดหยุ่นสูง

สรุป: Matplotlib เป็นเครื่องมือช่วยแสดงข้อมูลเป็นภาพได้อย่างมีประสิทธิภาพ เหมาะกับงานวิเคราะห์ทุกรูปแบบ

---

กรุณาตอบคำถามต่อไปนี้ในรูปแบบเดียวกัน:
""")

# สร้าง Prompt ที่สมบูรณ์
final_prompt = prompt_template.format(context=short_context, question=question)

# ตั้งค่า LLM
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
    api_key=OPENAI_API_KEY,
    streaming=True ,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# ฟังก์ชันประมวลผลแบบ async
async def run_async_query(prompt):
    response = await llm.ainvoke(prompt)
    return response.content

# ประมวลผลและวัดเวลา
start_q = time.perf_counter()
response = asyncio.run(run_async_query(final_prompt))
end_q = time.perf_counter()
response_time_1 = end_q - start_q

# แสดงผล
# print("คำตอบ:", response)

จากข้อมูลที่วิเคราะห์ได้ สามารถสรุปเกี่ยวกับการเปลี่ยนตำแหน่ง Legend ใน Matplotlib ได้ดังนี้:  
1. การเปลี่ยนตำแหน่ง Legend สามารถทำได้โดยการใช้พารามิเตอร์ 'loc' ในฟังก์ชัน ax.legend()  
2. ค่าที่สามารถใส่ใน 'loc' ได้แก่ 'upper left', 'upper right', 'lower left', 'lower right' ซึ่งแทนตำแหน่งที่ต้องการวาง Legend  
3. ตัวอย่างการเปลี่ยนตำแหน่ง Legend ไปยัง 'upper left' คือ ax.legend(loc='upper left', frameon=False)  
4. ถ้าต้องการเปลี่ยนตำแหน่ง Legend ไปยังตำแหน่งอื่น ๆ สามารถเปลี่ยนค่าใน 'loc' ได้ตามต้องการ

สรุป: การเปลี่ยนตำแหน่ง Legend ใน Matplotlib สามารถทำได้ง่ายๆ ผ่านพารามิเตอร์ 'loc' ในฟังก์ชัน ax.legend() ทำให้สามารถปรับตำแหน่งของ Legend ให้เหมาะสมกับกราฟของเราได้.

In [92]:
print(f"⏱ ใช้เวลาในการตอบ: {response_time_1:.2f} วินาที")

⏱ ใช้เวลาในการตอบ: 17.40 วินาที


Pattern 2

In [93]:
# โหลด tokenizer จาก BERT
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# โหลดโมเดล LaBSE สำหรับการสร้าง embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')


def reduce_vector_dimension(vec, target_dim):
    if vec.ndim == 1:
        vec = vec.reshape(1, -1)

    current_dim = vec.shape[1]
    if current_dim == target_dim:
        return vec.flatten()

    if vec.shape[0] == 1:
        # กรณี sample เดียว ให้เลือกตัดหรือ padding
        if current_dim > target_dim:
            reduced = vec[:, :target_dim]
        else:
            pad_width = target_dim - current_dim
            reduced = np.pad(vec, ((0, 0), (0, pad_width)), mode='constant')
    else:
        # มีหลาย sample ใช้ PCA
        pca = PCA(n_components=target_dim)
        reduced = pca.fit_transform(vec)

    return reduced.flatten()



# ฟังก์ชัน cosine similarity ที่รองรับเวกเตอร์ 1D
def cosine_similarity_2(vec1, vec2):
    vec1 = np.array(vec1).flatten()
    vec2 = np.array(vec2).flatten()

    # ตรวจสอบขนาดของเวกเตอร์
    if vec1.shape != vec2.shape:
        raise ValueError(f"Shape mismatch: {vec1.shape} vs {vec2.shape}")

    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0  # ป้องกันหารด้วยศูนย์

    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


# ฟังก์ชันลดจำนวน token โดยใช้ tokenizer ของ BERT
def reduce_token_with_bert(text, max_tokens=512):
    encoded = bert_tokenizer(text, truncation=True, max_length=max_tokens, return_tensors='pt')
    # แปลง token กลับเป็นข้อความหลังตัด token แล้ว
    truncated_text = bert_tokenizer.decode(encoded["input_ids"][0], skip_special_tokens=True)
    return truncated_text


In [94]:
async def retrieve_context_from_mongodb(question: str, top_k: int = 50):
    # สร้าง embedding สำหรับคำถามด้วย LaBSE
    question_vector = labse_model.encode([question], convert_to_numpy=True)

    # ตรวจสอบขนาดของ question_vector
    print(f"Question vector shape: {question_vector.shape}")

    # ดึงข้อมูลจาก MongoDB
    documents = list(collection.find())
    similarities = []
    
    # คำนวณ cosine similarity สำหรับแต่ละเอกสาร
    for doc in documents:
        doc_embedding = np.array(doc["embedding"]).flatten()

        # ตรวจสอบขนาดของ doc_embedding
        print(f"Document embedding shape: {doc_embedding.shape}")

        # ปรับขนาดเวกเตอร์ของ doc["embedding"] ให้ตรงกับขนาดของ question_vector
        target_dim = question_vector.shape[1]  # ใช้ขนาดของ question_vector
        if doc_embedding.shape[0] != target_dim:
            doc_embedding = reduce_vector_dimension(doc_embedding, target_dim)

        # ตรวจสอบขนาดของ doc_embedding หลังจากลดขนาด
        print(f"Reduced document embedding shape: {doc_embedding.shape}")

        # คำนวณ cosine similarity
        score = cosine_similarity_2(question_vector, doc_embedding)
        similarities.append((score, doc))

    # จัดเรียงตามคะแนน similarity
    similarities.sort(reverse=True, key=lambda x: x[0])

    # คืนค่าข้อความที่ถูกลดจำนวนโทเค็น
    reduced_texts = []
    for score, doc in similarities[:top_k]:
        reduced = reduce_token_with_bert(doc["raw_text"])
        reduced_texts.append(reduced)

    return "\n".join(reduced_texts)


In [95]:
# question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
question = "ขอตัวอย่างการเปลี่ยนตำแหน่ง Legend"
context = await retrieve_context_from_mongodb(question)
num_tokens_context = count_tokens_2(context, model="sentence-transformers/LaBSE")
print(f"จำนวนโทเค็นใน context: {num_tokens_context}")

Question vector shape: (1, 768)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: (768,)
Document embedding shape: (1536,)
Reduced document embedding shape: 

In [96]:
num_tokens_context = count_tokens_2(context, model="sentence-transformers/LaBSE")
print(f"จำนวนโทเค็นใน context: {num_tokens_context}")
# print(context)

จำนวนโทเค็นใน context: 512


In [97]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")

def reduce_context(text, num_tokens_context):
    tokens = encoding.encode(text)
    tokens = tokens[:num_tokens_context]
    return encoding.decode(tokens)

In [101]:
# ตัด context
context_trimmed = reduce_context(context, num_tokens_context)

# สร้าง PromptTemplate
prompt_template = PromptTemplate.from_template("""
ข้อมูลต่อไปนี้ถูกรวบรวมมาจากหลายแหล่งไฟล์ เช่น PDF, Word, Excel หรือ CSV ซึ่งอาจอยู่ในรูปแบบข้อความทั่วไปหรือเป็นข้อมูลเชิงตาราง:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- ไม่ต้องเขียนคำถามซ้ำ
- ไม่ต้องขึ้นต้นด้วยคำว่า "คำตอบ:"
- เริ่มต้นด้วยประโยค เช่น "จากเอกสารที่อ่าน..." หรือ "จากข้อมูลที่วิเคราะห์ได้..."
- จากนั้นจัดคำตอบให้อ่านง่ายในรูปแบบข้อ ๆ
- มีสรุปท้ายที่ใช้ถ้อยคำกระชับและไม่ซ้ำกับรายละเอียดด้านบน
- หลีกเลี่ยงการตอบซ้ำหรือลอกเนื้อหาเดิมซ้ำหลายรอบ

ตัวอย่างที่ 1:
จากข้อมูลที่วิเคราะห์ได้ สามารถสรุปเกี่ยวกับ Matplotlib ได้ดังนี้:  
1. เป็นไลบรารี่ในภาษา Python สำหรับสร้างภาพกราฟ 2 มิติ และ 3 มิติ  
2. รองรับการสร้างกราฟเส้น แท่ง วงกลม ฯลฯ  
3. ติดตั้งด้วยคำสั่ง pip install matplotlib  
4. เรียกใช้งานผ่าน import matplotlib.pyplot as plt  
5. ใช้งานง่ายและมีความยืดหยุ่นสูง

สรุป: Matplotlib เป็นเครื่องมือช่วยแสดงข้อมูลเป็นภาพได้อย่างมีประสิทธิภาพ เหมาะกับงานวิเคราะห์ทุกรูปแบบ

---

กรุณาตอบคำถามต่อไปนี้ในรูปแบบเดียวกัน:
""")

# สร้าง Prompt ที่สมบูรณ์
final_prompt = prompt_template.format(context=context_trimmed, question=question)

# ตั้งค่า LLM
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
    api_key=OPENAI_API_KEY,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)



# ฟังก์ชันประมวลผลแบบ async
async def run_async_query(prompt):
    response = await llm.ainvoke(prompt)
    return response.content

# ประมวลผลและวัดเวลา
start_q = time.perf_counter()
response = asyncio.run(run_async_query(final_prompt))
end_q = time.perf_counter()
response_time_2 = end_q - start_q

# แสดงผล
# print("คำตอบ:", response)


จากข้อมูลที่วิเคราะห์ได้ สามารถสรุปเกี่ยวกับการเปลี่ยนตำแหน่ง Legend ได้ดังนี้:  
1. Legend คือ สัญลักษณ์ที่ใช้แสดงความหมายของสี รูปแบบ หรือสัญลักษณ์อื่น ๆ ที่ใช้ในกราฟ
2. ใน Matplotlib สามารถเปลี่ยนตำแหน่งของ Legend ได้โดยใช้คำสั่ง plt.legend(loc='ตำแหน่งที่ต้องการ')
3. ตำแหน่งที่สามารถกำหนดได้มีดังนี้: 'upper left', 'upper right', 'lower left', 'lower right', 'center left', 'center right', 'lower center', 'upper center', 'center'
4. นอกจากนี้ยังสามารถกำหนดตำแหน่งของ Legend ได้โดยใช้พิกัด x และ y ด้วยคำสั่ง plt.legend(bbox_to_anchor=(x, y))

สรุป: การเปลี่ยนตำแหน่งของ Legend ใน Matplotlib สามารถทำได้ง่ายๆ ผ่านคำสั่ง plt.legend() ที่มีอาร์กิวเมนต์สำหรับกำหนดตำแหน่งที่ต้องการ

In [99]:
print(f"⏱ ใช้เวลาในการตอบ: {response_time_2:.2f} วินาที")

⏱ ใช้เวลาในการตอบ: 14.20 วินาที
